In [ ]:
%load_ext autoreload
%autoreload 2
from data_api import *
from tqdm import tqdm

In [ ]:
CONF_PATH = 'D:/CSE6242/TreeMusicRecommendation/Data/conf.json'

In [ ]:
database = database_api(CONF_PATH)
last_fm = lastfm_api(CONF_PATH)
result = last_fm.get_user_friends(username='rj')

In [ ]:
"construrt userdatabse with 100000 users, Do not run it again"
import queue
start_username = "Kimgs"
user_queue = queue.Queue()
user_queue.put(start_username)
users_processed = set()
users_processed.add(start_username)

while not user_queue.empty() and len(users_processed) < 10000:
    current_username = user_queue.get()
    friends = last_fm.get_user_friends(current_username)

    if friends:
        for friend in friends:
            friend_name = friend['name']
            if friend_name not in users_processed:
                users_processed.add(friend_name)
                user_queue.put(friend_name)
                database.save_users([friend])  # Assuming save_users takes a list
                
                # If we've reached 10,000 users, we can break early
                if len(users_processed) >= 10000:
                    break

print(f"Total users in database: {len(users_processed)}")


In [ ]:
# 备份 user 表
# cursor = database.cnx_cursor
# source_table = 'Users'
# new_table = 'Users_copy'

# drop the table if it exists
# drop_table_query = f"DROP TABLE IF EXISTS {new_table};"
# cursor.execute(drop_table_query)

# SQL statement to create a new table as a copy of another
# create_table_query = f"CREATE TABLE {new_table} LIKE {source_table};"
# cursor.execute(create_table_query)

# SQL statement to copy all data from the source table to the new table
# insert_data_query = f"INSERT INTO {new_table} SELECT * FROM {source_table};"
# cursor.execute(insert_data_query)

# Commit the changes
# database.cnx.commit()


In [ ]:
cursor = database.cnx_cursor
source_table = 'Users_copy'
new_table = 'Users'

# SQL statement to copy all data from the source table to the new table
insert_data_query = f"INSERT INTO {new_table} SELECT * FROM {source_table};"
cursor.execute(insert_data_query)

# Commit the changes
database.cnx.commit()


In [ ]:
# construct top_tracks
users = database.get_all_users()
for j in range(0, len(users)):
    try:
        user_id = users[j][0]
        user_name = users[j][1]
        top_tracks = last_fm.get_top_tracks(username=user_name)
        artist_infos = []
        if not top_tracks:
            print(j)
            continue
        for i in range(len(top_tracks)):
                track = top_tracks[i]
                url = track['track_url']
                start = len("https://www.last.fm/music/")
                end = url.index("/_/")
                artist_name = url[start:end]

                top_tracks[i]['artist_name'] =  artist_name
                artist_url = "https://www.last.fm/music/" + artist_name
                artist_info = {"artist_name": artist_name, "artist_url": artist_url}
                artist_infos.append(artist_info)
                top_tracks[i]['track_id'] = track['artist_name'] + ": " + track['track_name']

        
        database.save_artists(artist_infos)
        database.save_tracks(top_tracks)
        database.save_top_tracks(user_id, top_tracks)
    except:
        print('write user '+str(j)+' failed')
        continue


In [ ]:
# Construct listening history
users = database.get_all_users()
for j in range(len(users)):
    user_id = users[j][0]
    user_name = users[j][1]
    recent_tracks = last_fm.get_recent_tracks(username=user_name)
    if not recent_tracks:
        continue
    for i in range(len(recent_tracks)):
        try:
            track = recent_tracks[i]
            url = track['track_url']
            start = len("https://www.last.fm/music/")
            end = url.index("/_/")
            artist_name = url[start:end]

            recent_tracks[i]['artist_name'] =  artist_name
            artist_url = "https://www.last.fm/music/" + artist_name
            artist_info = {"artist_name": artist_name, "artist_url": artist_url}
            artist_infos.append(artist_info)
            recent_tracks[i]['track_id'] = track['artist_name'] + ": " + track['track_name']

        except:
            continue

    database.save_artists(artist_infos)
    database.save_tracks(recent_tracks)
    database.save_listening_history(1, recent_tracks)
    print(j)

In [ ]:
# copy top_tracks to top_tracks_copy
cursor = database.cnx_cursor
source_table = 'Top_track'
new_table = 'Top_track_copy'

# drop the table if it exists
drop_table_query = f"DROP TABLE IF EXISTS {new_table};"
cursor.execute(drop_table_query)

# SQL statement to create a new table as a copy of another
create_table_query = f"CREATE TABLE {new_table} LIKE {source_table};"
cursor.execute(create_table_query)

# SQL statement to copy all data from the source table to the new table
insert_data_query = f"INSERT INTO {new_table} SELECT * FROM {source_table};"
cursor.execute(insert_data_query)


In [ ]:
# test
user_id = 1
user_name = "rj"

top_artists = last_fm.get_top_artist(username=user_name)


# save top artists
database.save_artists(top_artists)
database.save_top_artists(user_id, top_artists)

In [ ]:
# save top artists
users = database.get_all_users()

for j in range(len(users)):
    try:
        user_id = users[j][0]
        user_name = users[j][1]
        top_artists = last_fm.get_top_artist(username=user_name)
        database.save_artists(top_artists)
        database.save_top_artists(user_id, top_artists)
        print("insert user "+str(j)+" success")
    except:
        print("insert user "+str(j)+" failed")
        continue


In [ ]:
# construct top artists


In [ ]:
all_tracks = database.get_all_tracks()
all_artists = database.get_all_artists()
print(len(all_tracks))

In [ ]:
# Don't run this cell
query = """TRUNCATE TABLE Track_tag"""

database.cnx_cursor.execute(query)

In [ ]:
test_str = "Heard It Everyday "
len(test_str.encode('utf-8')) <= len(test_str)

In [ ]:
for track in tqdm(all_tracks):
    track_tags = last_fm.get_track_tags(track[1], track[3])
    if track_tags is None or len(track_tags) == 0:
        continue
    tag_names = [t["tag_name"] for t in track_tags if len(t["tag_name"]) <= 30]
    database.save_tags(tag_names)
    tag_ids = database.get_tag_id(tag_names)
    tag_ids_with_count = []
    for ind, tag in enumerate(track_tags):
        tag_ids_with_count.append({"tag_id": tag_ids[ind], "tag_count": tag["tag_count"]})
    database.save_track_tag(track[0], tag_ids_with_count)

In [ ]:
query = """create table Artist_tag (
	tag_id BIGINT NOT NULL,
    artist_name VARCHAR(100) NOT NULL,
    tag_count INTEGER NOT NULL,
    foreign key (tag_id) references Tags(tag_id),
    foreign key (artist_name) references Artists(artist_name)
)"""

database.cnx_cursor.execute(query)

In [ ]:
# Don't run this cell
query = """TRUNCATE TABLE Artist_tag"""

database.cnx_cursor.execute(query)

In [ ]:
for artist in tqdm(all_artists):
    artist_tags = last_fm.get_artist_tags(artist[0])
    if artist_tags is None or len(artist_tags) == 0:
        continue
    tag_names = [t["tag_name"] for t in artist_tags if len(t["tag_name"]) <= 30]
    database.save_tags(tag_names)
    tag_ids = database.get_tag_id(tag_names)
    tag_ids_with_count = []
    for ind, tag in enumerate(artist_tags):
        tag_ids_with_count.append({"tag_id": tag_ids[ind], "tag_count": tag["tag_count"]})
    database.save_artist_tag(artist[0], tag_ids_with_count)